## GuardSQL

In [13]:
import json

import openai
import pandas as pd

import guardrails as gd
from utils import get_examples

from utils import sqllite_db_to_prompt, sqllite_db_to_prompt_tables

In [29]:
rail_spec = """
<rail version="0.1">

<output>
    <sql
        name="generated_sql"
        description="Generate SQL for the given natural language instruction."
        format="bug-free-sql"
        on-fail-bug-free-sql="reask" 
    />
</output>


<prompt>

Here are few text to sql examples:
{{examples}}

Generate a valid SQL query for the following natural language instruction:

{{nl_instruction}}

Here's schema about the database that you can use to generate the SQL query.
Try to avoid using joins if the data can be retrieved from the same table.

{{db_info}}

@complete_json_suffix
</prompt>


</rail>
"""

In [30]:
guard = gd.Guard.from_rail_string(rail_spec)

In [ ]:
dataset = []

with open('/Users/shreyarajpal/Downloads/spider/train_spider.json') as f:
    data = json.load(f)
    for row in data:
        dataset.append({
            'db_id': row['db_id'],
            'query': row['query'],
            'question': row['question']}
        )

df = pd.DataFrame(dataset)

In [ ]:
df

In [ ]:
from utils import get_examples

In [71]:
db_id = "table_10875694_11"


In [46]:
' '.join(['Query: {s0} \n JSON Object: "generated_sql": {s1}]'.format(s0=el[0], s1=el[1]) for el in get_examples('some', k=10)])

'Query: Tell me what the notes are for South Australia  \n JSON Object: "generated_sql": SELECT  col5 FROM table WHERE col3 = SOUTH AUSTRALIA] Query: What is the format for South Australia? \n JSON Object: "generated_sql": SELECT  col2 FROM table WHERE col0 = South Australia] Query: who is the manufacturer for the order year 1998? \n JSON Object: "generated_sql": SELECT  col1 FROM table WHERE col0 = 1998] Query: how many times is the fleet series (quantity) is 468-473 (6)? \n JSON Object: "generated_sql": SELECT COUNT col0 FROM table WHERE col3 = 468-473 (6)] Query: what is the fuel propulsion where the fleet series (quantity) is 310-329 (20)? \n JSON Object: "generated_sql": SELECT  col5 FROM table WHERE col3 = 310-329 (20)] Query: Name the background colour for the Australian Capital Territory \n JSON Object: "generated_sql": SELECT  col1 FROM table WHERE col0 = Australian Capital Territory] Query: how many times is the fuel propulsion is cng? \n JSON Object: "generated_sql": SELECT 

In [68]:
def nl2sql(nl_instruction: str, db_id: str) -> str:
    db_info = sqllite_db_to_prompt_tables(f'./data/train.db', table=db_id)

    #with open(f'/Users/shreyarajpal/Downloads/spider/database/{db_id}/schema.sql') as f:
    #    db_schema = f.read()
    examples = ['Query: {s0} \n JSON Object: KARSAS"generated_sql": {s1}KARKAS2 \n'.format(s0=el[0], s1=el[1]) for el in get_examples('some', k=10)]
    examples = [e.replace("KARSAS", "{").replace("KARKAS2", "}") for e in examples]

    response = guard(
        openai.Completion.create,
        prompt_params={
            'nl_instruction': nl_instruction,
            'examples': ' '.join(examples),
            'db_info': db_info,
            # 'db_schema': db_schema,
        },
        engine='text-davinci-003',
        temperature=0.0,
        max_tokens=512,
    )
    return response[1]['generated_sql']
query = nl2sql('what is the name of', db_id = "None")

/opt/conda/lib/python3.10/site-packages/eliot/json.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.  (This may have returned Python scalars in past versions.
  if isinstance(o, (numpy.bool, numpy.bool_)):


In [90]:
from lib.dbengine import DBEngine
import sqlite3
db = sqlite3.connect(dbfile)


def execute(instruction, dbfile = "./data/train.db", db_id = "table_1_1000181_1"):
    engine = DBEngine(dbfile)
    query = nl2sql(instruction, db_id = db_id)
    el = pd.read_sql_query(query, db)
    return el

execute("I need smth")



/opt/conda/lib/python3.10/site-packages/eliot/json.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.  (This may have returned Python scalars in past versions.
  if isinstance(o, (numpy.bool, numpy.bool_)):


,col0,col1,col2,col3,col4,col5
0,australian capital territory,blue/white,yaa·nna,act · celebration of a century 2013,yil·00a,slogan screenprinted on plate
1,new south wales,black/yellow,aa·nn·aa,new south wales,bx·99·hi,no slogan on current series
2,new south wales,black/white,aaa·nna,nsw,cpx·12a,optional white slimline series
3,northern territory,ochre/white,ca·nn·aa,nt · outback australia,cb·06·zz,new series began in june 2011
4,queensland,maroon/white,nnn·aaa,queensland · sunshine state,999·tlg,slogan embossed on plate
5,south australia,black/white,snnn·aaa,south australia,s000·azd,no slogan on current series
6,victoria,blue/white,aaa·nnn,victoria - the place to be,zzz·562,current series will be exhausted this year


In [88]:
print(guard.state.most_recent_call.history[-1].validated_output)

{'generated_sql': 'SELECT col1 FROM table_1_1000181_1'}


In [51]:
import os
os.environ['OPENAI_API_KEY'] = openai.api_key = ''


/opt/conda/lib/python3.10/site-packages/eliot/json.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.  (This may have returned Python scalars in past versions.
  if isinstance(o, (numpy.bool, numpy.bool_)):


TypeError: 'NoneType' object is not subscriptable

NameError: name 'engine' is not defined

In [ ]:
df_sample = df[:10]
df_sample

In [ ]:
# Apply a function to each row of the dataframe for the first 10 rows
# Add the output of this function as  a new column in the dataframe

df_sample['generated_sql'] = df_sample.apply(lambda row: nl2sql(row['question'], row['db_id']), axis=1)

# output = df[:1].apply(lambda row: nl2sql(row['question'], row['db_id']), axis=1)

In [ ]:
# Display only the columns question and generated_sql for the first 10 rows

df_sample[['question', 'generated_sql', 'query']]



Here are few text to sql examples:
Query: Tell me what the notes are for South Australia  
 JSON Object: "generated_sql": SELECT  col5 FROM table WHERE col3 = SOUTH AUSTRALIA] Query: What is the format for South Australia? 
 JSON Object: "generated_sql": SELECT  col2 FROM table WHERE col0 = South Australia] Query: who is the manufacturer for the order year 1998? 
 JSON Object: "generated_sql": SELECT  col1 FROM table WHERE col0 = 1998] Query: how many times is the fleet series (quantity) is 468-473 (6)? 
 JSON Object: "generated_sql": SELECT COUNT col0 FROM table WHERE col3 = 468-473 (6)] Query: what is the fuel propulsion where the fleet series (quantity) is 310-329 (20)? 
 JSON Object: "generated_sql": SELECT  col5 FROM table WHERE col3 = 310-329 (20)] Query: Name the background colour for the Australian Capital Territory 
 JSON Object: "generated_sql": SELECT  col1 FROM table WHERE col0 = Australian Capital Territory] Query: how many times is the fuel propulsion is cng? 
 JSON Obj